# Bilik Torch

## Imports

In [1]:
import os
import sys
import glob
import torch
import shutil
import pickle
import mlflow
import logging
import argparse
import datetime
import commentjson
import numpy as np
import scipy.signal
import tensorflow as tf
from bunch import Bunch
from pathlib import Path
from random import randint
import torch.functional as F
from torch.nn.functional import pad
from collections import OrderedDict
logger = logging.getLogger("logger")


2023-06-26 14:16:50.177330: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 14:16:50.723144: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Config functions

### define Config class

In [2]:
CONFIG_VERBOSE_WAIVER = ['save_model', 'tracking_uri', 'quiet', 'sim_dir', 'train_writer', 'test_writer', 'valid_writer']
class Config(Bunch):
    """ class for handling dicrionary as class attributes """

    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)

    def print(self):
        line_len = 122
        line = "-" * line_len
        logger.info(line + "\n" +
              "| {:^35s} | {:^80} |\n".format('Feature', 'Value') +
              "=" * line_len)
        for key, val in sorted(self.items(), key= lambda x: x[0]):
            if isinstance(val, OrderedDict):
                raise NotImplementedError("Nested configs are not implemented")
            else:
                if key not in CONFIG_VERBOSE_WAIVER:
                    logger.info("| {:35s} | {:80} |\n".format(key, str(val)) + line)
        logger.info("\n")


### Define arguments

In [3]:
def get_args(argv):
    argparser = argparse.ArgumentParser(description=__doc__)
    argparser.add_argument('--config', default=None, type=str, help='path to config file')
    argparser.add_argument('--seed', default=None, type=int, help='randomization seed')
    argparser.add_argument('--exp_name', default=None, type=int, help='Experiment name')
    argparser.add_argument('--num_targets', default=None, type=int, help='Number of simulated targets')
    argparser.set_defaults(quiet=False)
    args, unknown = argparser.parse_known_args(argv)
    #args = argparser.parse_args()

    return args


### Read the config file

In [4]:
def read_json_to_dict(fname):
    """ read json config file into ordered-dict """
    fname = Path(fname)
    with fname.open('rt') as handle:
        config_dict = commentjson.load(handle, object_hook=OrderedDict)
        return config_dict
    

In [5]:
def read_config(args):
    """ read config from json file and update by the command line arguments """
    if args.config is not None:
        json_file = args.config
    else:
        json_file = "/home/leshkar/Desktop/BGU/configs/config.json"  # Replace with your default config file path

    config_dict = read_json_to_dict(json_file)
    config = Config(config_dict)

    for arg in sorted(vars(args)):
        key = arg
        val = getattr(args, arg)
        if val is not None:
            setattr(config, key, val)

    if args.seed is None and config.seed is None:
        
        MAX_SEED = sys.maxsize
        config.seed = randint(0, MAX_SEED)

    return config


### GPU initialization

In [6]:
def gpu_init():
    """Allows GPU memory growth"""
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        torch.backends.cudnn.benchmark = True
        torch.backends.cuda.autotune = True
        torch.backends.cudnn.enabled = True

    return device


### Logger and tracker

In [7]:
def set_logger_and_tracker(config):
    ''' configure the mlflow tracker:
        1. set tracking location (uri)
        2. configure exp name/id
        3. define parameters to be documented
    '''

    config.exp_name_time = "{}_{}_{}".format(config.exp_name,datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),config.seed)
    config.tensor_board_dir = os.path.join('..',
                                           'results',
                                           config.exp_name,
                                           config.exp_name_time)

    if not os.path.exists(config.tensor_board_dir):
        os.makedirs(config.tensor_board_dir)


### Save scripts

In [8]:
def save_scripts(config,SRC_DIR):
    path = os.path.join(config.tensor_board_dir, 'scripts')
    if not os.path.exists(path):
        os.makedirs(path)
    scripts_to_save = glob.glob('{}/**/*.py'.format(SRC_DIR), recursive=True) + [config.config]
    scripts_to_save = [script for script in scripts_to_save if '{}/results'.format(SRC_DIR) not in script]
    if scripts_to_save is not None:
        for script in scripts_to_save:
            dst_file = os.path.join(path, os.path.basename(script))
            try:
                shutil.copyfile(os.path.join(os.path.dirname(sys.argv[0]), script), dst_file)
            except:
                print()

In [9]:
def print_config(config):
    print('')
    print('#' * 70)
    print('Configurations at beginning of run')
    print('#' * 70)
    for key in config.keys():
        print('{}, {}'.format(key,config['{}'.format(key)]))
    print('')
    print('')

## Activate Conf

In [10]:
SRC_DIR = os.getcwd()
config_path = '/home/leshkar/Desktop/BGU/configs/config.json'  
args = get_args(config_path)
config = read_config(args)
gpu_init()
set_logger_and_tracker(config)
#save_scripts(config,SRC_DIR)
print_config(config)



######################################################################
Configurations at beginning of run
######################################################################
model_input_dim, [None]
model_output_dim, 1
quiet, False
seed, 6477700957040699125
exp_name, temp
load_complete_model, False
load_model_path, 
con_inf_rng_path, 
con_inf_vel_path, 
eval_model_pth, 
detection_pfa_miss_M_valid, 5000
detection_exp_type, pd
ipix_pkl_path, 
ipix_pkl_path_dir, /home/leshkar/Desktop/RD_NN-main/datasets/IPIX/15m/pkl/hh
ipix_pkl_cv_hold_out, 
ipix_cdf_files_list, []
ipix_skip_cv_iters, []
ipix_predefined_cv_iters, []
ipix_cv_mode, False
ipix_cv_script, main_train
ipix_cv_rng_pth, ../results/IPIX_3m_HH_K64_8targets_CV_twostage_fc_rng/IPIX_3m_HH_K64_8targets_CV_twostage_fc_rng_2022-03-12_17-29-13_405075
ipix_cv_vel_pth, ../results/IPIX_3m_HH_K64_8targets_CV_twostage_fc_vel/IPIX_3m_HH_K64_8targets_CV_twostage_fc_vel_2022-03-12_09-02-17_826518
sweep_run_eval_con_inf, False
sweep_run_eval, T

## Data Loading

In [11]:
def gen_compound_gaussian_pipeline_dataset(config, M0, M1):

    def gen_frame2d(ind):
        """
        1.  generate single fast-time x slow-time clutter matrix.
            Each row is i.i.d by compound gaussian distribution distribution
            config.sigma_f = clutter correlation factor, (how much the clutter is correlated between pulses)
            clutter_fd = clutter Doppler frequency
        2.  generate single fast-time x slow-time white gaussian noise matrix.
        3.  generate single fast-time x slow-time target echo signal matrix.
        """
        # generate clutter signal
        if config.compound_gaussian_single_clutter_vel:
            if config.compound_gaussian_constant_clutter_vel is not None:
                clutter_vel = tf.constant([config.compound_gaussian_constant_clutter_vel], dtype=tf.float32)
            else:
                clutter_vel = tf.random.uniform([1], np.floor(config.v_r_min), np.floor(config.v_r_max))
            clutter_fd = tf.cast(2 * np.pi * config.T_PRI * ((2 * config.f_c * clutter_vel) / 3e8), dtype=tf.complex128)
            C = tf.math.exp(-2 * (config.sigma_f ** 2) * (np.pi ** 2) * (C_cords ** 2) - 1j * C_cords * clutter_fd)
        else:
            assert config.compound_gaussian_dims == 2
            raise Exception('')
        z_t = tf.cast(tf.random.normal([K, N_range_bins], mean=0.0, stddev=np.sqrt(0.5)), dtype=tf.complex128) + \
              1j * tf.cast(tf.random.normal([K, N_range_bins], mean=0.0, stddev=np.sqrt(0.5)), dtype=tf.complex128)
        e, V = tf.linalg.eigh(C)
        e_sqrt = tf.math.sqrt(tf.math.maximum(tf.math.real(e), 0.0))
        E = tf.cast(tf.linalg.diag(e_sqrt), dtype=tf.complex128)
        A = tf.matmul(V, E)
        # [K, N_range_bins]
        w_t = tf.matmul(A, z_t)
        # create texture s ~ gamma(shape)
        if config.compound_gaussian_random_gamma_shape:
            gamma_shape = tf.gather(tf.random.uniform([1, ], minval=config.gamma_shape_range[0], maxval=config.gamma_shape_range[1] + 0.001), 0)
        else:
            gamma_shape = config.gamma_shape
        s = tf.random.gamma([N_range_bins,], alpha=gamma_shape, beta=gamma_shape)
        c_t = tf.cast(tf.expand_dims(tf.math.sqrt(s), 0), dtype=tf.complex128) * w_t
        # "insert" clutter Doppler vector for each range bin
        clutter_omega_r = ((2 * np.pi) / N) * tf.range(N_range_bins, dtype=tf.float32)
        clutter_range_steering_tensor = tf.math.exp(-1j * tf.cast(tf.expand_dims(tf.range(N, dtype=tf.float32), -1) * tf.expand_dims(clutter_omega_r, 0), dtype=tf.complex128))
        c_tensor = tf.matmul(c_t, tf.transpose(clutter_range_steering_tensor))
        c_tensor = tf.transpose(c_tensor)

        # generate WGN
        n_tensor = tf.cast(tf.random.normal([K, N], mean=0.0, stddev=np.sqrt(0.5 * sigma2)), dtype=tf.complex128) + \
              1j * tf.cast(tf.random.normal([K, N], mean=0.0, stddev=np.sqrt(0.5 * sigma2)), dtype=tf.complex128)
        n_tensor = tf.transpose(n_tensor)

        # clutter velocity reconstruction label tensor
        clutter_inds_vel = tf.expand_dims(tf.math.argmin(tf.abs(tf.expand_dims(clutter_vel, 1) - tf.expand_dims(recon_vec_vel, 0)), axis=1), 1)
        clutter_label_tensor = tf.scatter_nd(tf.expand_dims(clutter_inds_vel, 1), tf.expand_dims(tf.ones_like(clutter_inds_vel), 1), (recon_vec_vel.shape[0], 1))

        if not with_target:
            param_val_tensor = tf.ones(config.num_targets) * -1000.0, tf.ones(config.num_targets) * -1000.0
            return c_tensor + n_tensor, tf.zeros((recon_vec_rng.shape[0], recon_vec_vel.shape[0]), dtype=tf.int64), \
                   param_val_tensor, tf.ones(config.num_targets) * -1000.0, gamma_shape, clutter_vel, clutter_label_tensor
        else:
            # generate target signal
            # clutter plus noise energy is: N*K*(N_range_bins + sigma2)
            rd_signal, label_tensor, param_val_tensor, scnr_tensor = gen_target_matrix(config, tf.sqrt(N*K*(N_range_bins + sigma2)), clutter_vel, N, K, recon_vec_rng, recon_vec_vel)

            return rd_signal + c_tensor + n_tensor, label_tensor, param_val_tensor, scnr_tensor, gamma_shape, clutter_vel, clutter_label_tensor

    def get_tfds(M_tfds):
        tfds = tf.data.Dataset.range(0, M_tfds)
        if config.compound_gaussian_dims == 2:
            tfds = tfds.map(gen_frame2d, num_parallel_calls=-1)
        else:
            raise Exception(' ')
        return tfds

    # parameters
    assert config.compound_gaussian_dims == 2
    assert not (config.SCNR_db_random_constant and config.SCNR_db_random_choice)
    K = config.K
    N = config.N
    N_range_bins = N // 2
    config.r_0_max = (N_range_bins - 1) * (3e8 / (2.0 * config.B_chirp))

    # Clutter correlation matrix
    C_mesh = tf.meshgrid(tf.range(K), tf.range(K))
    C_cords = tf.cast(C_mesh[1] - C_mesh[0], dtype=tf.complex128)
    # WGN variance, determined by CNR_db
    CNR = 10.0 ** (config.CNR_db / 10.0)
    sigma2 = N_range_bins / CNR
    # reconstruction vector for label creation
    recon_vec_rng = tf.cast(get_reconstruction_point_cloud_vec(config, param_ind=0), dtype=tf.float32)
    recon_vec_vel = tf.cast(get_reconstruction_point_cloud_vec(config, param_ind=1), dtype=tf.float32)

    # get target and non-target tfds
    with_target = False
    tfds0 = get_tfds(M0) # [complex_tensor, label_tensor, gamma_shape]
    with_target = True
    _res = gen_frame2d(0)
    tfds1 = get_tfds(M1) # [complex_tensor, label_tensor, (rng_vals, vel_vals), SCNR_db, gamma_shape]
    assert tfds1.element_spec[0] == tfds0.element_spec[0] and tfds1.element_spec[1] == tfds0.element_spec[1]
    tfds = tfds1.concatenate(tfds0)
    tfds = tfds.map(split_auxillary_structure)

    return tfds


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

class CompoundGaussianPipelineDataset(Dataset):
    def __init__(self, config, M0, M1):
        self.config = config
        self.M0 = M0
        self.M1 = M1

    def __len__(self):
        return self.M0 + self.M1

    def __getitem__(self, index):
        if index < self.M0:
            with_target = False
        else:
            with_target = True

        # Implement the logic to generate the data item based on the index and with_target value

        return data_item

def get_dataset_compound_gaussian(config, apply_tf_preprocess_pipe=True):
    data = {}
    M1_train = config.M_train
    M0_train = int(config.M_train * config.without_target_ratio)
    M1_valid = config.M_valid
    M0_valid = int(config.M_valid * config.without_target_ratio_test)
    M1_test = config.M_test
    M0_test = int(config.M_test * config.without_target_ratio_test)

    if config.embedded_target:
        assert config.compound_gaussian_single_clutter_vel
    data['train'] = CompoundGaussianPipelineDataset(config, M0_train, M1_train)
    data['valid'] = CompoundGaussianPipelineDataset(config, M0_valid, M1_valid)
    data['test'] = CompoundGaussianPipelineDataset(config, M0_test, M1_test)

    if apply_tf_preprocess_pipe:
        for set_type in ['train', 'valid', 'test']:
            data[set_type] = torch_dataset_pipeline(config, data[set_type])

    return data


In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class WGNPipelineDataset(Dataset):
    def __init__(self, config, M0, M1):
        self.config = config
        self.M0 = M0
        self.M1 = M1

    def __len__(self):
        return self.M0 + self.M1

    def __getitem__(self, index):
        if index < self.M0:
            with_target = False
        else:
            with_target = True

        # Implement the logic to generate the data item based on the index and with_target value

        return data_item

def get_dataset_wgn(config, apply_tf_preprocess_pipe=True):
    data = {}
    M1_train = config.M_train
    M0_train = int(config.M_train * config.without_target_ratio)
    M1_valid = config.M_valid
    M0_valid = int(config.M_valid * config.without_target_ratio_test)
    M1_test = config.M_test
    M0_test = int(config.M_test * config.without_target_ratio_test)

    data['train'] = WGNPipelineDataset(config, M0_train, M1_train)
    data['valid'] = WGNPipelineDataset(config, M0_valid, M1_valid)
    data['test'] = WGNPipelineDataset(config, M0_test, M1_test)

    if apply_tf_preprocess_pipe:
        # Apply any necessary preprocessing using PyTorch transformations

    return data


IndentationError: expected an indented block after 'if' statement on line 36 (4145474748.py, line 39)

In [ ]:
def get_model_output_dim(data, set_type):
    if len(list(data['train'].element_spec[1].shape)) > 1:
        return [list(data['train'].element_spec[1].shape)[0]]
    if type(data[set_type].element_spec[1]) == type(tuple()):
        return list(data[set_type].element_spec[1][0].shape)
    else:
        return [list(data[set_type].element_spec[1].shape)[0]]

def get_model_input_dim(data, set_type):
    if isinstance(data[set_type].element_spec[0], tuple):
        model_input_dim = []
        for spec in data[set_type].element_spec[0]:
            model_input_dim.append(list(spec.shape))
    else:
        model_input_dim = list(data[set_type].element_spec[0].shape)

    return model_input_dim

In [ ]:
def make_iterators(data, config):
    M_train = len(data['train'])
    print('make_iterators(): M_train: {}'.format(M_train))
    train_dataset = data['train'].shuffle(M_train, reshuffle_each_iteration=True)
    train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, drop_last=True, shuffle=True, num_workers=config.num_workers, pin_memory=True)
    valid_iter = torch.utils.data.DataLoader(data['valid'], batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers, pin_memory=True)
    test_iter = torch.utils.data.DataLoader(data['test'], batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers, pin_memory=True)
    iterators = {'train': train_iter, 'valid': valid_iter, 'test': test_iter}
    return iterators


### Get Valid Bins (range & velocity)

***The function takes a config, a 2D array shape, range and velocity bins arrays. filtering the velocity and range bins based on minimum and maximum values defined in the config.***

***The function returns the filtered bin values along with their indices of the valid bins.***

In [ ]:
def get_valid_2d_bins(config, full_shape, range_bins_values, vel_bins_values):
    assert len(full_shape) == 2
    vel_bins_values = torch.tensor(vel_bins_values)
    range_bins_values = torch.tensor(range_bins_values)

    valid_vel_bins = torch.nonzero(
        (vel_bins_values >= config.v_0_min) & (vel_bins_values <= config.v_0_max)
    ).squeeze()

    if valid_vel_bins[-1] < full_shape[1] - 1:
        valid_vel_bins = torch.cat((valid_vel_bins, torch.tensor([valid_vel_bins[-1] + 1])))

    if valid_vel_bins[0] > 0:
        valid_vel_bins = torch.cat((torch.tensor([valid_vel_bins[0] - 1]), valid_vel_bins))

    vel_bins_values = vel_bins_values[valid_vel_bins]

    valid_range_bins = torch.nonzero(
        (range_bins_values >= config.r_0_min) & (range_bins_values <= config.r_0_max)
    ).squeeze()

    range_bins_values = range_bins_values[valid_range_bins]

    return [range_bins_values, vel_bins_values], [valid_range_bins, valid_vel_bins]


### SCNR

***gets the signal to clutter plus noise ratio:***
- if config.SCNR_db_random_choice, generates random integers between 0 and the length of config.SCNRs_eval. 
The SCNR values are gathered from SCNRs_eval using tf.gather() based on the indices in scnr_eval_inds
- else if config.SCNR_db_random_constant, a single random SCNR value is chosen from config.SCNRs_eval and repeated for targets_num times.
    scnr_eval_inds is generated to select a random index from config.SCNRs_eval.
    
    The selected SCNR value is multiplied by tf.ones(targets_num) to create a tensor of the same length as targets_num.
    The SCNR values are assigned to SCNR_db.
- if none of them so a constant SCNR value is used from config.SCNR_db, repeated for targets_num times.

    The constant SCNR value is multiplied by tf.ones(targets_num) to create a tensor of the same length as targets_num.***

In [ ]:
def torch_get_SCNR_db(config, targets_num):
    if config.SCNR_db_random_choice:
        SCNRs_eval = torch.tensor(config.SCNRs_eval, dtype=torch.float32)
        scnr_eval_inds = torch.randint(0, len(config.SCNRs_eval), size=(targets_num,))
        SCNR_db = SCNRs_eval[scnr_eval_inds]
    elif config.SCNR_db_random_constant:
        SCNRs_eval = torch.tensor(config.SCNRs_eval, dtype=torch.float32)
        scnr_eval_inds = torch.randint(0, len(config.SCNRs_eval), size=(1,))
        SCNR_db = SCNRs_eval[scnr_eval_inds] * torch.ones(targets_num)
    elif config.random_SCNR:
        SCNR_db = torch.FloatTensor(targets_num).uniform_(config.SCNR_db_range[0], config.SCNR_db_range[1] + 0.001)
    else:
        SCNR_db = torch.tensor(config.SCNR_db, dtype=torch.float32) * torch.ones(targets_num)
    
    return SCNR_db

### Split auxillary

In [ ]:
def split_auxillary_structure(mat_complex, mat_label, param_val, scnr, gamma_shape, clutter_vel, clutter_label_tensor):
    return torch.squeeze(mat_complex), mat_label, (param_val, scnr, gamma_shape, clutter_vel, clutter_label_tensor)

### Preprocessing pipeline

In [ ]:
def torch_dataset_pipeline(config, data):
    def two_stage_fc_preprocess(mat_complex, label):
        mat = cube_center_and_reshape(mat_complex)
        return mat, label
    
    def two_stage_fc_stdize(mat_complex, mat_label, aux):
        mat_complex = mat_complex / mat_complex.std(dim=0, keepdim=True).to(torch.complex128)
        return mat_complex, mat_label, aux
    
    def two_stage_fc_preprocess_cg(mat_complex, mat_label, aux):
        mat_complex, mat_label = two_stage_fc_preprocess(mat_complex, mat_label)
        return mat_complex, mat_label, aux
    
    def transpose_mat_complex(mat_complex, mat_label, aux):
        mat_complex = mat_complex.transpose(0, 1)
        return mat_complex, mat_label, aux
    
    def concat_real_imag_cg(mat_complex, mat_label, aux):
        return torch.cat((mat_complex.real, mat_complex.imag), dim=-1), mat_label, aux
    
    def preprocess_label2d(mat_label):
        reduce_axis = 1 if config.estimation_params == ["rng"] else 0
        mat_label = mat_label.sum(dim=reduce_axis).clamp(0, 1).to(torch.float32)
        return mat_label
    
    def cg_preprocess_label_2dims(mat_complex, mat_label, aux):
        mat_label = preprocess_label2d(mat_label)
        return mat_complex, mat_label, aux
    
    def cg_preprocess_label_1dim(mat_complex, mat_label, aux):
        mat_label = mat_label.squeeze().clamp(0, 1).to(torch.float32)
        return mat_complex, mat_label, aux
    
    def cube_center_and_reshape(mat):
        mat_center = mat - mat.view(-1, mat.shape[-1]).mean(dim=0)
        return mat_center.view(-1, mat_center.shape[-1])
    
    if config.data_name == "ipix":
        if config.model_name == "Detection-TwoStage-FC":
            assert config.estimation_params == ["rng"] or config.estimation_params == ["vel"]
            if config.estimation_params == ["rng"]:
                data = data.map(transpose_mat_complex)
            data = data.map(two_stage_fc_preprocess_cg)
            if config.two_stage_fc_stdize:
                data = data.map(two_stage_fc_stdize)
            data = data.map(concat_real_imag_cg)
            data = data.map(cg_preprocess_label_2dims)

    if config.data_name == "compound_gaussian" or config.data_name == "wgn":
        if config.model_name == "Detection-TwoStage-FC":
            assert config.estimation_params == ["rng"] or config.estimation_params == ["vel"]
            if config.estimation_params == ["rng"]:
                data = data.map(transpose_mat_complex)
            data = data.map(two_stage_fc_preprocess_cg)
            if config.two_stage_fc_stdize:
                data = data.map(two_stage_fc_stdize)
            data = data.map(concat_real_imag_cg)
            if config.compound_gaussian_dims == 2:
                data = data.map(cg_preprocess_label_2dims)
            else:
                data = data.map(cg_preprocess_label_1dim)
        elif config.model_name == "Detection-FC":
            data = data.map(lambda t, label, aux: (torch.cat((t.real, t.imag), dim=0).squeeze(),
                                                   label.squeeze().clamp(0, 1).to(torch.float32), aux))
    
    return data

### Target and label tensors generator
***This function generates a target matrix along with corresponding labels, parameter values, and SCNR tensors***

In [ ]:
def gen_target_matrix(config, cn_norm, clutter_vel_local, N, K, recon_vec_rng, recon_vec_vel):

    # randomly determine the number of targets or set it to a constant value
    if config.random_num_targets:
        targets_num = torch.randint(1, config.num_targets + 1, (1,))
    else:
        targets_num = torch.tensor([config.num_targets], dtype=torch.int64)
    
    # Generate target velocities within specified range considering the presence of embedded targets
    if config.embedded_target:
        targets_vel = torch.empty(targets_num).uniform_(max(clutter_vel_local - config.embedded_target_vel_offset, config.v_0_min),
                                                     min(clutter_vel_local + config.embedded_target_vel_offset, config.v_0_max))
    else:
        targets_vel = torch.empty(targets_num).uniform_(config.v_0_min, config.v_0_max)
    
    # Generate target ranges within specified range
    targets_rng = torch.empty(targets_num).uniform_(config.r_0_min, config.r_0_max)
    
    # compute doppler and range target frequencies
    targets_omega_d = torch.tensor(2 * np.pi * config.T_PRI * ((2 * config.f_c * targets_vel) / 3e8), dtype=torch.complex128)
    targets_omega_r = torch.tensor(2 * np.pi * ((2 * config.B_chirp * targets_rng) / (3e8 * N)), dtype=torch.complex128)
    
    # compute doppler and range steering tensors
    doppler_steering_tensor = torch.exp(-1j * targets_omega_d.unsqueeze(1) * torch.arange(K, dtype=torch.complex128))
    range_steering_tensor = torch.exp(-1j * targets_omega_r.unsqueeze(1) * torch.arange(N, dtype=torch.complex128))
    
    # compute range-doppler signal and get the SCNR
    rd_signal = range_steering_tensor.unsqueeze(2) * doppler_steering_tensor.unsqueeze(1)
    SCNR_db = get_SCNR_db(config, targets_num)  # Assuming get_SCNR_db is defined
    
    # Adjust phase of the range-Doppler signal based on the configuration
    if config.signal_random_phase:
        rd_signal = rd_signal * torch.exp(1j * torch.empty(targets_num).uniform_(0, 2 * np.pi).unsqueeze(1).unsqueeze(1))
    elif config.signal_physical_phase:
        targets_tau0 = (2 * targets_rng) / 3e8
        rd_signal = rd_signal * torch.exp(1j * (torch.tensor(-2 * np.pi * config.f_c * targets_tau0 + np.pi * (config.B_chirp / (config.N * config.f_s)) * (targets_tau0 ** 2), dtype=torch.complex128)).unsqueeze(1).unsqueeze(1))

    # compensate for the appropriate SCNR level
    s_norm = torch.norm(rd_signal, dim=[1, 2]).real
    sig_amp = (10 ** (SCNR_db.float() / 20.0)) * (cn_norm.float() / s_norm)
    rd_signal = torch.sum(sig_amp.unsqueeze(-1).unsqueeze(-1) * rd_signal, dim=0)
    
    # gen label vector
    trgt_inds_vel = torch.argmin(torch.abs(targets_vel.unsqueeze(1) - recon_vec_vel.unsqueeze(0)), dim=1).unsqueeze(1)
    trgt_inds_rng = torch.argmin(torch.abs(targets_rng.unsqueeze(1) - recon_vec_rng.unsqueeze(0)), dim=1).unsqueeze(1)
    trgt_inds = torch.cat((trgt_inds_rng, trgt_inds_vel), dim=1)
    label_tensor = torch.scatter(torch.zeros((recon_vec_rng.shape[0], recon_vec_vel.shape[0]), dtype=torch.float32), 0, trgt_inds, torch.ones_like(trgt_inds_vel).squeeze(1))

    # gen parameter value and SCNR tensor
    param_val_tensor = (torch.cat((targets_rng, torch.ones(config.num_targets - targets_num) * -1000.0), dim=0),
                    torch.cat((targets_vel, torch.ones(config.num_targets - targets_num) * -1000.0), dim=0))
    scnr_tensor = torch.cat((SCNR_db, torch.ones(config.num_targets - targets_num) * -1000.0), dim=0)

    return rd_signal, label_tensor, param_val_tensor, scnr_tensor


### fft resolution
calculates the resolutions and values for the range, velocity, and azimuth dimensions of the point cloud

In [ ]:

def get_fft_resolutions(config, data_shape, T_PRI=None):
    assert len(data_shape) == 4
    T_PRI = data_shape[1] * (1 / config.f_s) + config.T_idle if T_PRI is None else T_PRI
    vel_res = 3e8 / (2 * config.f_c * data_shape[2] * T_PRI)
    range_res = 3e8 / (2 * config.B_chirp)
    range_bins_values = torch.tensor([range_res * (i - data_shape[1] // 2) for i in range(data_shape[1])])
    vel_bins_values = torch.tensor([vel_res * (i - data_shape[2] // 2) for i in range(data_shape[2])])

    azimuth_bins_values = torch.tensor(np.arcsin([(2.0 * (i - data_shape[3] // 2)) / data_shape[3]
                                          for i in range(data_shape[3])]) * (180.0 / np.pi))

    return range_res, vel_res, range_bins_values, vel_bins_values, azimuth_bins_values


### point cloud reconstraction
calculates and returns vectors used for point cloud reconstruction in the ipix pipeline. 
the function first checks if we use FFT dimensions for point cloud reconstruction, If this option is enabled, it rescales the dimensions. This rescaling is done to achieve higher resolution in the reconstructed point cloud.

The function then calls the get_fft_resolutions function to calculate the resolution and values for the range, velocity, and azimuth dimensions of the point cloud.

If param_ind is 0, it returns the range bins values, which represent the distances from the radar sensor to the objects in the range dimension. If param_ind is 1, it returns the velocity bins values, representing the velocities of the objects.

In [ ]:
def get_reconstruction_point_cloud_vec(config, param_ind):

    if config.point_cloud_reconstruction_fft_dims:
        N = config.point_cloud_reconstruction_fft_dim_factor * config.N
        config.B_chirp = config.point_cloud_reconstruction_fft_dim_factor * config.B_chirp  # multiply to rescale range dimension
        K = config.point_cloud_reconstruction_fft_dim_factor * config.K
        L = config.point_cloud_reconstruction_fft_dim_factor * config.L

        range_res, vel_res, recon_vec_rng, recon_vec, azimuth_bins_values = get_fft_resolutions(config, [1, N, K, L], T_PRI=config.T_PRI)
        bin_values_list, valid_bins_list = get_valid_2d_bins(config, [N, K], recon_vec_rng, recon_vec)
        range_bins_values = bin_values_list[0]
        vel_bins_values = bin_values_list[1]

        config.B_chirp = config.B_chirp / config.point_cloud_reconstruction_fft_dim_factor
        if param_ind == 0:
            return torch.tensor(range_bins_values)
        elif param_ind == 1:
            return torch.tensor(vel_bins_values)
        else:
            raise Exception('  ')
    else:
        raise Exception('get_reconstruction_point_cloud_res(): Unsupported...')


### dataset pipeline generator
generates a pipeline dataset for the "ipix" scenario. It processes the "ipix" data frames, applies random Doppler shifts (if enabled), generates radar signals and labels for target detection, and returns a dataset consisting of frames with and without targets

In [ ]:
def gen_ipix_pipeline_dataset(c_tensor_total, clutter_vel, config, M0, M1):
    def gen_ipix_frame2d(ind):
        c_tensor = c_tensor_total[:config.N, :config.K]
        clutter_vel_local = clutter_vel
        if config.ipix_random_shift_doppler:
            shift_min = clutter_vel_local - config.v_r_min
            shift_max = config.v_r_max - clutter_vel_local
            doppler_shift_v = torch.FloatTensor(1).uniform_(-shift_min, shift_max)
            clutter_vel_local = clutter_vel + doppler_shift_v.item()
            shift_factor = torch.exp(-1j * 2 * np.pi * ((2 * config.f_c * doppler_shift_v) / 3e8) * config.T_PRI * torch.arange(c_tensor.shape[1]).to(torch.complex128))
            c_tensor = c_tensor * shift_factor

        if with_target is False:
            param_val_tensor = (torch.ones(config.num_targets) * -1000.0, torch.ones(config.num_targets) * -1000.0)
            return c_tensor, torch.zeros((recon_vec_rng.shape[0], recon_vec_vel.shape[0]), dtype=torch.int64), \
                   param_val_tensor, torch.ones(config.num_targets) * -1000.0, torch.tensor(0.0), clutter_vel_local, torch.tensor(0.0)
        else:
            cn_norm = torch.abs(torch.norm(c_tensor))
            rd_signal, label_tensor, param_val_tensor, scnr_tensor = gen_target_matrix(config, cn_norm, clutter_vel_local, N, K, recon_vec_rng, recon_vec_vel)

            return rd_signal + c_tensor, label_tensor, param_val_tensor, scnr_tensor, torch.tensor(0.0), clutter_vel_local, torch.tensor(0.0)

    def get_ipix_tfds(M_tfds):
        _res = gen_ipix_frame2d(0)
        tfds = [gen_ipix_frame2d(i) for i in range(M_tfds)]
        return tfds

    assert config.N == c_tensor_total.shape[0]
    assert not (config.SCNR_db_random_constant and config.SCNR_db_random_choice)
    N = config.N
    K = config.K

    recon_vec_rng = torch.tensor(get_reconstruction_point_cloud_vec(config, param_ind=0), dtype=torch.float32)
    recon_vec_vel = torch.tensor(get_reconstruction_point_cloud_vec(config, param_ind=1), dtype=torch.float32)

    with_target = False
    tfds0 = get_ipix_tfds(M0)
    with_target = True
    tfds1 = get_ipix_tfds(M1)

    assert tfds1[0][0].shape == tfds0[0][0].shape and tfds1[0][1].shape == tfds0[0][1].shape
    tfds = tfds1 + tfds0
    tfds = [split_auxillary_structure(item) for item in tfds]

    return tfds


### ipix data reader
reads and preprocesses  the ipix data from the files. It extracts relevant parameters, truncates the data based on the provided configuration, computes the clutter velocity and generates the processed data tensor.

In [ ]:

def read_data_ipix(config):
    with open(config.ipix_pkl_path, 'rb') as handle:
        ipix_data = pickle.load(handle)
        PRI = ipix_data['PRI']
        B = ipix_data['B']
        rng_bins = ipix_data['rng_bins']
        adc_data = ipix_data['adc_data']

    rng_bins = rng_bins[:config.ipix_max_nrange_bins]
    adc_data = adc_data[:config.ipix_max_nrange_bins, :]
    if not config.ipix_file_range_bins:
        assert config.N <= len(rng_bins) * 2
        adc_data = adc_data[:config.N // 2, :]
        rng_bins = rng_bins[:config.N // 2]
    else:
        config.N = len(rng_bins) * 2
    assert config.N == len(rng_bins) * 2
    config.T_PRI = PRI
    config.B_chirp = B
    if '19980205_191043' in config.ipix_pkl_path:
        # cut weird zero part of this file
        adc_data = adc_data[:, :50000]
    # convert to fast-time x slow-time data
    rng_bins = rng_bins - rng_bins[0]
    config.r_0_max = rng_bins[-1]
    clutter_omega_r = ((2 * np.pi) / config.N) * ((2 * B) / 3e8) * rng_bins
    # workaround to prevent GPU overflow in multiple iterations
    try:
        clutter_range_steering_tensor = torch.exp(-1j * torch.tensor(torch.unsqueeze(torch.arange(config.N, dtype=torch.float32), -1) * torch.unsqueeze(clutter_omega_r, 0), dtype=torch.complex128))
        c_tensor = clutter_range_steering_tensor @ adc_data
    except:
        clutter_range_steering_tensor = np.exp(-1j * torch.tensor(torch.unsqueeze(torch.arange(config.N, dtype=torch.float32), -1) * torch.unsqueeze(clutter_omega_r, 0), dtype=torch.complex128))
        c_tensor = clutter_range_steering_tensor @ adc_data


    """
    estimate clutter Doppler frequency using welch method: 
        c_tensor[i] = e ^ {j 2 \pi f_d kT_PRI}
        f_d = (2 * f_c * clutter_vel) / c
    """

    Pxx_den_list = []
    for i in range(adc_data.shape[0]):
        f, Pxx_den = scipy.signal.welch(adc_data[i], 1 / PRI, return_onesided=False)
        Pxx_den_list.append(Pxx_den)

    PSD = np.mean(np.array(Pxx_den_list), 0)
    PSD = PSD / np.max(PSD)
    clutter_fd = f[np.argmax(PSD)]
    clutter_vel = -(3e8 * clutter_fd) / (2 * 9.39e9)

    return c_tensor, rng_bins, clutter_vel


### create dataset
This function generates the dataset. It loads and processes data from multiple files, splits it into train, validation, and test datasets, and applies the preprocessing pipeline.

In [ ]:
def get_dataset_ipix(config, apply_tf_preprocess_pipe=True, split_data=True, return_dict=False):
    if config.ipix_cv_mode:
        # load train and validation
        data_dict_per_file = {}
        cdf_files_list = [f for f in os.listdir(config.ipix_pkl_path_dir) if not f.startswith('.')]
        cdf_files_list = [x for x in cdf_files_list if x not in config.ipix_pkl_cv_hold_out]
        for cdf_file in cdf_files_list:
            config.ipix_pkl_path = os.path.join(config.ipix_pkl_path_dir, cdf_file)
            # read raw data and convert to fast-time x slow-time complex data
            c_tensor_total, rng_bins_ipix, clutter_vel = read_data_ipix(config)
            config.r_0_max = rng_bins_ipix[-1]
            M_valid = int(config.M_valid / len(cdf_files_list))
            M0_valid = int(config.without_target_ratio_test * M_valid)
            M1_valid = M_valid
            M_train = int(config.M_train / len(cdf_files_list))
            M0_train = int(config.without_target_ratio * M_train)
            M1_train = M_train
            # generate tf.data.Dataset objects
            data = {}
            c_tensor_total_valid = c_tensor_total[:, int(0.9 * c_tensor_total.shape[1]):]
            data['valid'] = gen_ipix_pipeline_dataset(c_tensor_total_valid, clutter_vel, config, M0_valid, M1_valid)

            c_tensor_total_train = c_tensor_total[:, :int(0.9 * c_tensor_total.shape[1])]
            data['train'] = gen_ipix_pipeline_dataset(c_tensor_total_train, clutter_vel, config, M0_train, M1_train)

            if apply_tf_preprocess_pipe:
                # add data pipeline functions (maps)
                for set_type in ['train', 'valid']:
                    data[set_type] = torch_dataset_pipeline(config, data[set_type])

            data_dict_per_file[cdf_file] = data

        data = {}
        for set_type in ['train', 'valid']:
            data[set_type] = data_dict_per_file[cdf_files_list[0]][set_type]
            for cdf_file in cdf_files_list[1:]:
                data[set_type] = data[set_type].concatenate(data_dict_per_file[cdf_file][set_type])

        # load test
        data_dict_per_file = {}
        for cdf_file in config.ipix_pkl_cv_hold_out:
            assert cdf_file not in cdf_files_list
            config.ipix_pkl_path = os.path.join(config.ipix_pkl_path_dir, cdf_file)
            c_tensor_total_test, rng_bins_ipix, clutter_vel = read_data_ipix(config)
            M_test = int(config.M_test / len(config.ipix_pkl_cv_hold_out))
            M0_test = int(config.without_target_ratio_test * M_test)
            M1_test = M_test
            data['test'] = gen_ipix_pipeline_dataset(c_tensor_total_test, clutter_vel, config, M0_test, M1_test)

            if apply_tf_preprocess_pipe:
                data['test'] = torch_dataset_pipeline(config, data['test'])

            data_dict_per_file[cdf_file] = data['test']

        data['test'] = data_dict_per_file[config.ipix_pkl_cv_hold_out[0]]
        for cdf_file in config.ipix_pkl_cv_hold_out[1:]:
            data['test'] = data['test'].concatenate(data_dict_per_file[cdf_file])
    else:
        data_dict_per_file = {}
        cdf_files_list = [f for f in os.listdir(config.ipix_pkl_path_dir) if not f.startswith('.')]
        assert len(cdf_files_list) > 0
        for cdf_file in cdf_files_list:
            config.ipix_pkl_path = os.path.join(config.ipix_pkl_path_dir, cdf_file)
            c_tensor_total, rng_bins_ipix, clutter_vel = read_data_ipix(config)
            config.r_0_max = rng_bins_ipix[-1]
            M_test = int(config.M_test / len(cdf_files_list))
            M0_test = int(config.without_target_ratio_test * M_test)
            M1_test = M_test
            M_valid = int(config.M_valid / len(cdf_files_list))
            M0_valid = int(config.without_target_ratio_test * M_valid)
            M1_valid = M_valid
            M_train = int(config.M_train / len(cdf_files_list))
            M0_train = int(config.without_target_ratio * M_train)
            M1_train = M_train

            data = {}
            if split_data:
                c_tensor_total_test = c_tensor_total[:, int(0.9 * c_tensor_total.shape[1]):]
                data['test'] = gen_ipix_pipeline_dataset(c_tensor_total_test, clutter_vel, config, M0_test, M1_test)

                c_tensor_total_valid = c_tensor_total[:, int(0.85 * c_tensor_total.shape[1]): int(0.9 * c_tensor_total.shape[1])]
                data['valid'] = gen_ipix_pipeline_dataset(c_tensor_total_valid, clutter_vel, config, M0_valid, M1_valid)

                c_tensor_total_train = c_tensor_total[:, :int(0.85 * c_tensor_total.shape[1])]
                data['train'] = gen_ipix_pipeline_dataset(c_tensor_total_train, clutter_vel, config, M0_train, M1_train)
            else:
                M1_test = config.M_test
                M0_test = int(config.M_test * config.without_target_ratio_test)
                data['test'] = gen_ipix_pipeline_dataset(c_tensor_total, clutter_vel, config, M0_test, M1_test)

            if apply_tf_preprocess_pipe:
                for set_type in ['train', 'valid', 'test']:
                    data[set_type] = torch_dataset_pipeline(config, data[set_type])

            data_dict_per_file[cdf_file] = data

        if return_dict:
            return data_dict_per_file
        else:
            data = {}
            for set_type in ['train', 'valid', 'test']:
                data[set_type] = data_dict_per_file[cdf_files_list[0]][set_type]
                for cdf_file in cdf_files_list[1:]:
                    data[set_type] = data[set_type].concatenate(data_dict_per_file[cdf_file][set_type])

    return data

### load data main func
loads and preprocesses data based on the provided configuration, supports multiple data types, sets the model input and output dimensions, and optionally returns data iterators.

In [ ]:
def load_data(config, use_make_iterators=True, apply_tf_preprocess_pipe=True):
    model_input_dim_set = 'train'
    if config.data_name == "ipix":
        data = get_dataset_ipix(config, apply_tf_preprocess_pipe=apply_tf_preprocess_pipe)
        model_input_dim_set = 'test'
    elif config.data_name == "compound_gaussian":
        data = get_dataset_compound_gaussian(config, apply_tf_preprocess_pipe=apply_tf_preprocess_pipe)
        if config.compound_gaussian_add_wgn:
            data_wgn = get_dataset_wgn(config, apply_tf_preprocess_pipe=apply_tf_preprocess_pipe)
            for key in data.keys():
                # rd_signal , label_tensor, (param_val_tensor, scnr_tensor, gamma_shape, clutter_vel, clutter_label_tensor)
                data[key] = data[key].map(lambda x0, x1, x2: (x0, x1, (x2[0], x2[1], x2[2], torch.tensor(0.0), torch.tensor(0.0))))
                data[key] = data[key].concatenate(data_wgn[key])
    elif config.data_name == "wgn":
        data = get_dataset_wgn(config, apply_tf_preprocess_pipe=apply_tf_preprocess_pipe)
    else:
        raise Exception(' ')

    # set model_input_dim
    config.model_input_dim = get_model_input_dim(data, model_input_dim_set)
    config.model_output_dim = get_model_output_dim(data, model_input_dim_set)

    if use_make_iterators:
        # make data iterators (shuffle, batch, etc.)
        data_iterators = make_iterators(data, config)
        return config, data_iterators
    else:
        return config, data


## Activations

### Config

In [ ]:
SRC_DIR = os.getcwd()
config_path = '/home/leshkar/Desktop/BGU/configs/config.json'  
args = get_args(config_path)
config = read_config(args)
gpu_init()
set_logger_and_tracker(config)
#save_scripts(config,SRC_DIR)
print_config(config)

### Data loading

In [ ]:
import scipy
config, data = load_data(config)